In [10]:
!pip install -q transformers accelerate datasets torch scikit-learn pandas


In [11]:
import pandas as pd
import numpy as np
import torch

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [35]:
df = pd.read_csv("/Reviews.csv")

df = df[["Text", "Score"]].sample(10000, random_state=42)

def label_sentiment(score):
    if score <= 2:
        return 0   # Negative
    elif score == 3:
        return 1   # Neutral
    else:
        return 2   # Positive

df["label"] = df["Score"].apply(label_sentiment)


In [36]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)


In [37]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=128
)


In [38]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)


In [39]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3
).to(device)


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [40]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="weighted"
    )
    acc = accuracy_score(labels, predictions)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


In [41]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    learning_rate=2e-5,
    save_strategy="no",
    fp16=True
)


In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.401944,0.849000,0.818196,0.814903,0.849000
2,0.395141,0.384400,0.851500,0.836162,0.830199,0.851500


TrainOutput(global_step=500, training_loss=0.39514068603515623, metrics={'train_runtime': 47.4504, 'train_samples_per_second': 337.194, 'train_steps_per_second': 10.537, 'total_flos': 529879044096000.0, 'train_loss': 0.39514068603515623, 'epoch': 2.0})

Accuracy: 0.845 → 84.5% F1-score: 0.805 Validation Loss: 0.45


A simple real world check..

In [44]:
text = "The food tasted awful and was completely spoiled."

inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128
).to(device)

outputs = model(**inputs)
prediction = outputs.logits.argmax().item()

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
print("Sentiment:", label_map[prediction])


Sentiment: Negative


In [53]:
text = "The food was rotten."

inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128
).to(device)

outputs = model(**inputs)
model.eval()

prediction = outputs.logits.argmax().item()

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
print("Sentiment:", label_map[prediction])


Sentiment: Negative


In [48]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('my_model/tokenizer_config.json', 'my_model/tokenizer.json')